In [1]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import json

import evaluate

import numpy as np
import pandas as pd

from nltk.tokenize import sent_tokenize

from datasets import Dataset, load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, EvalPrediction
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from typing import Any, Dict, Union

In [3]:
tokenizer = T5Tokenizer.from_pretrained("ai-forever/ruT5-base")
model = T5ForConditionalGeneration.from_pretrained("ai-forever/ruT5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/lib/python3/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
# metrics:
bleu4 = evaluate.load("bleu")
sbleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
tokenizer.eos_token_id

2

In [6]:
tokenizer.pad_token_id

0

In [7]:
with open("tf_dataset_pretty_filtered.json", 'r', encoding="utf8") as inp:
    tf_dataset = json.load(inp)

tf_dataset_train, tf_dataset_val, tf_dataset_test = tf_dataset["train"], tf_dataset["val"], tf_dataset["test"]
tf_dataset_train = Dataset.from_list(tf_dataset_train)
tf_dataset_val = Dataset.from_list(tf_dataset_val)
tf_dataset_test = Dataset.from_list(tf_dataset_test)

In [8]:
len(tf_dataset_train), len(tf_dataset_val), len(tf_dataset_test)

(3288, 175, 187)

In [8]:
tf_dataset_train[0]

{'example_id': 'high15596.txt',
 'article': 'YUZHOU, HENAN -An accident in a central China coal mine killed 21 miners Saturday and left another 16 trapped underground , the government said.\nThe death\nrose to 26 Sunday morning as rescuers were battling to reach the 11 miners who were still trapped underground, rescue headquarters said.\nRescuers were battling to reach the 11 miners still trapped underground, but chances for them to survive were very slim, said Du Bo, deputy chief of the rescue headquarters.\n"Based upon past experience, the remaining 11 miners could be buried in coal dust, so the survival chances are frail," Du said.\nMore than 2,500 tons of coal dust smothered  the pit after the gas leak , which hampered  the rescue, said Du.\nThe gas outburst happened at 6:03 a.m. Saturday when 276 miners were working underground in the mine in Yuzhou City. A total of 239 workers escaped but 21 were found dead and 16 trapped.\nAn initial  investigation showed that 173,500 cubic mete

In [9]:
option_id_dict = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3
}

def to_new_format(example: dict[str, Union[str, list[str]]]) -> str:
  inp, label = '', ''
  example["options_ru"] = [option for option in example["options_ru"] if option]
  right_answer = example['options_ru'][option_id_dict[example['answer']]]

  right_answer = right_answer.replace('"', "'")

  qtext_orig = example["question"].lower()
  if ("not true" in qtext_orig) or ("false" in qtext_orig) or ("n't true" in qtext_orig) or ("untrue" in qtext_orig):
      if ("not false" in qtext_orig) or ("n't false" in qtext_orig):
          inp += example['article_ru'] + " " + "ВОПРОС: Какое высказывание СООТВЕТСТВУЕТ тексту? "
      else:
          inp += example['article_ru'] + " " + "ВОПРОС: Какое высказывание НЕ СООТВЕТСТВУЕТ тексту? "
  else:
      inp += example['article_ru'] + " " + "ВОПРОС: Какое высказывание СООТВЕТСТВУЕТ тексту? "

  inp += f'ПРАВИЛЬНЫЙ ОТВЕТ: "{right_answer}".'
  inp += 'НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА: '

  options = example["options_ru"]
  options = [
      option.replace('"', "'") for option in options if option != right_answer
  ]
  options = [
      f'"{option}"' for option in options
  ]
  label = "; ".join(options)
    
  return {"inp": inp, "distractors": label}

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inp"]
    )
    labels = tokenizer(
        examples["distractors"]
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tf_dataset_train = tf_dataset_train.map(to_new_format)
tf_dataset_val = tf_dataset_val.map(to_new_format)
tf_dataset_test = tf_dataset_test.map(to_new_format)

Map:   0%|          | 0/3288 [00:00<?, ? examples/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

In [11]:
tf_dataset_train = tf_dataset_train.map(preprocess_function, batched=True, batch_size=2)
tf_dataset_val = tf_dataset_val.map(preprocess_function, batched=True, batch_size=2)
tf_dataset_test = tf_dataset_test.map(preprocess_function, batched=True, batch_size=2)

Map:   0%|          | 0/3288 [00:00<?, ? examples/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

In [12]:
BATCH_SIZE  = 1
NUM_TRAIN_EPOCHS = 20
MODEL_NAME="RuT5-RACE-tf-1"

args = Seq2SeqTrainingArguments(
    output_dir=MODEL_NAME,
    evaluation_strategy="epoch", save_strategy="epoch",
    # evaluation_strategy="steps", eval_steps=100, save_steps=100,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    prediction_loss_only=False,
    gradient_checkpointing=True,
    predict_with_generate=True, fp16=True,
    eval_accumulation_steps=1
)

In [13]:
tf_dataset_val[0]["labels"]

[49,
 28162,
 5442,
 1262,
 1740,
 1543,
 314,
 7225,
 9773,
 4,
 13386,
 49,
 4468,
 425,
 4550,
 151,
 6935,
 3061,
 3,
 110,
 296,
 69,
 688,
 188,
 4,
 13386,
 49,
 517,
 1543,
 2352,
 13,
 2080,
 4406,
 4,
 83,
 2]

In [14]:
def compute_metric_values(output: list[str], label_batch: list[str]) -> dict[str, Any]:
    metric_dict = {
        "bleu": bleu4.compute(predictions=output, references=[[label] for label in label_batch]),
        "sbleu": sbleu.compute(predictions=output, references=[[label] for label in label_batch]),
        "rouge": rouge.compute(predictions=output, references=label_batch),
        "meteor": meteor.compute(predictions=output, references=label_batch)
    }
    return metric_dict

def compute_metrics(eval_preds: EvalPrediction) -> dict[str, Any]:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids

    if isinstance(preds, tuple):
        preds = preds[0]

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels > 0, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    preds = [
        sent.replace("<pad>", " ").replace("</s>", " ").strip() for sent in preds
    ]
    labels = [
        sent.replace("<pad>", " ").replace("</s>", " ").strip() for sent in labels
    ]

    metrics = compute_metric_values(preds, labels)
    metric_dict = {
        "bleu": metrics["bleu"]["bleu"],
        "sbleu": metrics["sbleu"]["score"],
        "rouge1": metrics["rouge"]["rouge1"],
        "rouge2": metrics["rouge"]["rouge2"],
        "rougeL": metrics["rouge"]["rougeL"],
        "rougeLsum": metrics["rouge"]["rougeLsum"],
        "meteor": metrics["meteor"]["meteor"]
    }
    return metric_dict

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

In [16]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tf_dataset_train,
    eval_dataset=tf_dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Bleu,Sbleu,Rouge1,Rouge2,Rougel,Rougelsum,Meteor
1,2.381300,2.010943,0.059092,5.909222,0.015238,0.000000,0.014921,0.015873,0.186060
2,2.003800,1.970030,0.057546,5.754616,0.015365,0.002857,0.014730,0.014921,0.186020
3,1.792700,1.982938,0.063245,6.324499,0.021729,0.003810,0.021584,0.021143,0.191351
4,1.503300,2.052280,0.058028,5.802785,0.016852,0.003810,0.016952,0.017393,0.185798


/home/user/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/user/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/user/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/user/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `